In [1]:
import fasttext
import fasttext.util
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle('../data/filtered_nodes.pkl')

In [3]:
title_list = list(df['title'])
keywords_list = list(df['keywords'])

# fasttext

In [4]:
model = fasttext.load_model('cc.en.300.bin')

In [15]:
def word_embedding(model, keywords_list):
    embedding = [ np.array(list(map(model.get_word_vector, keywords))) for keywords in keywords_list ]
    return embedding

def sentence_embedding(model, sentence_list):
    embedding = [ model.get_sentence_vector(' '.join(sentence.split())) for sentence in sentence_list ]
    return embedding

In [ ]:
keywords_embedding_list = word_embedding(model, keywords_list)
keywords_embedding_mean_list = [ np.mean(keywords_embedding, axis=0) for keywords_embedding in keywords_embedding_list]
title_embedding_list = sentence_embedding(model, title_list)

# KeyBERT

In [19]:
from keybert import KeyBERT
from joblib import Parallel, delayed

In [21]:
abstract_list = list(df['abstract'])

In [22]:
def process_abstract(abstract, kw_model, threshold):
    keywords = kw_model.extract_keywords(abstract, top_n=20, use_mmr=True)
    filtered_keywords = [keyword[0] for keyword in keywords if keyword[1] > threshold]
    return filtered_keywords

In [23]:
threshold = 0.3
kw_model = KeyBERT()

# Parallel processing to extract and filter keywords for each abstract
keywords_from_abstract = Parallel(n_jobs=-1)(delayed(process_abstract)(abstract, kw_model, threshold) for abstract in abstract_list)

KeyboardInterrupt: 

In [ ]:
abstract_keywords_embedding_list = word_embedding(model, keywords_from_abstract)
abstract_keywords_embedding_mean_list = [ np.mean(keywords_embedding, axis=0) for keywords_embedding in abstract_keywords_embedding_list]